<a href="https://colab.research.google.com/github/StetskoSergey/pida5_stetsko/blob/master/aml6%20dz%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ДЗ - Требуется построить модель рекомендаций на основе скрытых факторов (implicit) на основе dataset’а https://grouplens.org/datasets/hetrec-2011/ (Delicious Bookmarks)” (edited)

In [0]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from implicit.als import AlternatingLeastSquares

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [0]:
user_taggedbookmarks = pd.read_csv('user_taggedbookmarks.dat', sep = '\t')

In [0]:
# построим рекомендации по количеству тагетов, для этого сгруппируем количество тагетов по Пользователям и Ссылкам
data = user_taggedbookmarks.groupby(['userID' , 'bookmarkID']).tagID.count().reset_index()

In [55]:
data.head()

,userID,bookmarkID,tagID
0,8,1,1
1,8,2,1
2,8,7,3
3,8,8,3
4,8,9,2


In [0]:
users = list(np.sort(data.userID.unique()))
bookmarks = list(np.sort(data.bookmarkID.unique()))

tags = list(data.tagID)

In [68]:
len(bookmarks)

69223

In [0]:
# нужно найти порядковые значения элемнтов в списке пользователей и буукмарков
rows = [users.index(i) for i in data.userID.values]
cols = [bookmarks.index(i) for i in data.bookmarkID.values]

In [73]:
rows[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [21]:
len(tags)

104799

In [0]:
data_sparse = csr_matrix((tags,(rows,cols)), shape=(len(users), len(bookmarks)))

In [81]:
model = AlternatingLeastSquares(factors = 50, use_gpu=False)
model.fit(data_sparse)

In [0]:
user = 0 # у нас номер 8 в списке

user_items = data_sparse.T.tocsc()

recomendations = model.recommend(0, user_items)



In [95]:
# список рекомендованных ссылок
[(bookmarks[i[0]], i[1]) for i in recomendations]

[(480, 0.054921634),
 (2031, 0.038050808),
 (1172, 0.037752423),
 (1947, 0.028631778),
 (2525, 0.02836818),
 (2535, 0.026642688),
 (1671, 0.02548683),
 (1707, 0.025444964),
 (1416, 0.024955615),
 (2554, 0.024872549)]